<a href="https://colab.research.google.com/github/jennkimerson/OCR_ArchivalDataOrganization_HGARC/blob/master/Mk_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mk. 3
Reads in text vertically in attempts to preserve white spaces. Then the text will be transposed into horizontal text with white spaces preserved.

Work in progress, last updated 5/1/2019

## I. Data Selection
Prompts the user to select pages they want to read in

In [1]:
!pip install PyPDF2

    100% |████████████████████████████████| 81kB 3.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [3]:
import PyPDF2

# takes in a file from the user and imports the image file
from google.colab import files

uploaded = files.upload()

Saving Bellah, James Warner inventory 1965-2016.pdf to Bellah, James Warner inventory 1965-2016.pdf


## II. File Conversion
Read in PDF to images

## III. Enhance Image
Reduce the background noise in images for accurate scanning

Credit to: 
https://gist.github.com/githubharald/24052afa05b6811b7f7a821fea1ef6e8#file-preprocesstextimage-py

In [0]:
import numpy as np
import cv2

In [0]:
# read
img = cv2.imread('sample.jpg', cv2.IMREAD_GRAYSCALE)

In [0]:
# increase contrast
pxmin = np.min(img)
pxmax = np.max(img)
imgContrast = (img - pxmin) / (pxmax - pxmin) * 255

In [0]:
# increase line width
kernel = np.ones((3, 3), np.uint8)
imgMorph = cv2.erode(imgContrast, kernel, iterations = 1)

In [0]:
# write
cv2.imwrite('out.jpg', imgMorph)

## III. Horizontal Scan

Horizontally scan the image and extract the text and create a text file

In [18]:
!pip install pytesseract
!sudo apt install tesseract-ocr
#!git clone https://github.com/jennkimerson/OCR_ArchivalDataOrganization_HGARC.git
import pytesseract

    100% |████████████████████████████████| 174kB 6.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/d5/90/56/ab7b652592da86821293f7cadc1c554aa376a0d57ce414d0a0
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 6 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0

In [0]:
try:
    from PIL import Image
except ImportError:
    import Image

In [20]:
# converts image file to string
#result = pytesseract.image_to_string(Image.open('out.jpg'))
text = pytesseract.image_to_string(Image.open('out.jpg'), config='-psm 6')

#print (result)
print ("text :{0}".format(text))

text :Box 5

Box 1
Box 5

folder - Book formt

folder - Misic score with TIS from P. 8. Odegard who d/d scores incl.

folder - clippings of reviews

folder - Preface to Japanese ed. +. 2 /. dated Se 196} and c. copy with
corrections indicated in other copy.

folder - Preliminary corr. Houghton 1955-56
incl. draft (holo.) of E B, telegram to (R. Wilson) at publishers
corr re its being BOM choice 1956 and copy of BOM announcement also
re its Eng. publ.

folder - contracts

folder - publicity

folder - fan mil

folder - movie corr.

folder - motion picture notes - holo. 18/.

folder - motion pict. script +t. c. on yellow paper.


In [0]:
# converts string to text file
with open('textfile.txt', 'w') as f:
  f.write(text)

files.download('textfile.txt')

## IV. Vertical Scan

Vertically scan the image and extract the text and create a text file

Credit to:
https://github.com/madmaze/pytesseract/blob/master/src/pytesseract.py

In [0]:
import os
import sys
import subprocess
import tempfile
import shlex
import string
from glob import iglob
from csv import QUOTE_NONE
from functools import wraps
from pkgutil import find_loader
from distutils.version import LooseVersion
from os.path import realpath, normpath, normcase

In [0]:
numpy_installed = find_loader('numpy') is not None
if numpy_installed:
    from numpy import ndarray

In [0]:
from io import BytesIO
pandas_installed = find_loader('pandas') is not None
if pandas_installed:
    import pandas as pd

In [0]:
tesseract_cmd = 'tesseract'
RGB_MODE = 'RGB'
OSD_KEYS = {
    'Page number': ('page_num', int),
    'Orientation in degrees': ('orientation', int),
    'Rotate': ('rotate', int),
    'Orientation confidence': ('orientation_conf', float),
    'Script': ('script', str),
    'Script confidence': ('script_conf', float)
}

In [0]:
class Output:
    BYTES = 'bytes'
    DATAFRAME = 'data.frame'
    DICT = 'dict'
    STRING = 'string'


In [0]:
class PandasNotSupported(EnvironmentError):
    def __init__(self):
        super(PandasNotSupported, self).__init__('Missing pandas package')

In [0]:
class TesseractError(RuntimeError):
    def __init__(self, status, message):
        self.status = status
        self.message = message
        self.args = (status, message)

In [0]:
class TesseractNotFoundError(EnvironmentError):
    def __init__(self):
        super(TesseractNotFoundError, self).__init__(
            tesseract_cmd + " is not installed or it's not in your path"
        )

In [0]:
class TSVNotSupported(EnvironmentError):
    def __init__(self):
        super(TSVNotSupported, self).__init__(
            'TSV output not supported. Tesseract >= 3.05 required'
        )


In [0]:
def run_once(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        if wrapper._result is wrapper:
            wrapper._result = func(*args, **kwargs)
        return wrapper._result

    wrapper._result = wrapper
    return wrapper

In [0]:
def get_errors(error_string):
    return u' '.join(
        line for line in error_string.decode('utf-8').splitlines()
    ).strip()

In [0]:
def cleanup(temp_name):
    ''' Tries to remove temp files by filename wildcard path. '''
    for filename in iglob(temp_name + '*' if temp_name else temp_name):
        try:
            os.remove(filename)
        except OSError:
            pass


In [0]:
def prepare(image):
    if isinstance(image, Image.Image):
        return image

    if numpy_installed and isinstance(image, ndarray):
        return Image.fromarray(image)

    raise TypeError('Unsupported image object')

In [0]:
def save_image(image):
    _, temp_name = tempfile.mkstemp(prefix='tess_')
    if isinstance(image, str):
        return temp_name, realpath(normpath(normcase(image)))

    image = prepare(image)
    img_extension = image.format
    if image.format not in {'JPEG', 'PNG', 'TIFF', 'BMP', 'GIF'}:
        img_extension = 'PNG'

    if not image.mode.startswith(RGB_MODE):
        image = image.convert(RGB_MODE)

    if 'A' in image.getbands():
        # discard and replace the alpha channel with white background
        background = Image.new(RGB_MODE, image.size, (255, 255, 255))
        background.paste(image, (0, 0), image)
        image = background

    input_file_name = temp_name + os.extsep + img_extension
    image.save(input_file_name, format=img_extension, **image.info)
    return temp_name, input_file_name

In [0]:
def subprocess_args(include_stdout=True):
    # See https://github.com/pyinstaller/pyinstaller/wiki/Recipe-subprocess
    # for reference and comments.

    kwargs = {
        'stdin': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'startupinfo': None,
        'env': os.environ
    }

    if hasattr(subprocess, 'STARTUPINFO'):
        kwargs['startupinfo'] = subprocess.STARTUPINFO()
        kwargs['startupinfo'].dwFlags |= subprocess.STARTF_USESHOWWINDOW
        kwargs['startupinfo'].wShowWindow = subprocess.SW_HIDE

    if include_stdout:
        kwargs['stdout'] = subprocess.PIPE

    return kwargs

In [0]:
def run_tesseract(input_filename,
                  output_filename_base,
                  extension,
                  lang,
                  config='',
                  nice=0):
    cmd_args = []

    if not sys.platform.startswith('win32') and nice != 0:
        cmd_args += ('nice', '-n', str(nice))

    cmd_args += (tesseract_cmd, input_filename, output_filename_base)

    if lang is not None:
        cmd_args += ('-l', lang)

    cmd_args += shlex.split(config)

    if extension not in {'box', 'osd', 'tsv'}:
        cmd_args.append(extension)

    try:
        proc = subprocess.Popen(cmd_args, **subprocess_args())
    except OSError:
        raise TesseractNotFoundError()

    try:
        _, error_string = proc.communicate()
    finally:
        proc.stdin.close()
        proc.stdout.close()
        proc.stderr.close()

    if proc.returncode:
        raise TesseractError(proc.returncode, get_errors(error_string))


In [0]:
def run_and_get_output(image,
                       extension,
                       lang=None,
                       config='',
                       nice=0,
                       return_bytes=False):

    temp_name, input_filename = '', ''
    try:
        temp_name, input_filename = save_image(image)
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name + '_out',
            'extension': extension,
            'lang': lang,
            'config': config,
            'nice': nice
        }

        run_tesseract(**kwargs)
        filename = kwargs['output_filename_base'] + os.extsep + extension
        with open(filename, 'rb') as output_file:
            if return_bytes:
                return output_file.read()
            return output_file.read().decode('utf-8').strip()
    finally:
        cleanup(temp_name)

In [0]:
def file_to_dict(tsv, cell_delimiter, str_col_idx):
    result = {}
    rows = [row.split(cell_delimiter) for row in tsv.split('\n')]
    if not rows:
        return result

    header = rows.pop(0)
    length = len(header)
    if len(rows[-1]) < length:
        # Fixes bug that occurs when last text string in TSV is null, and
        # last row is missing a final cell in TSV file
        rows[-1].append('')

    if str_col_idx < 0:
        str_col_idx += length

    for i, head in enumerate(header):
        result[head] = list()
        for row in rows:
            if len(row) <= i:
                continue

            val = row[i]
            if row[i].isdigit() and i != str_col_idx:
                val = int(row[i])
            result[head].append(val)

    return result

In [0]:
def is_valid(val, _type):
    if _type is int:
        return val.isdigit()

    if _type is float:
        try:
            float(val)
            return True
        except ValueError:
            return False

    return True

In [0]:
def osd_to_dict(osd):
    return {
        OSD_KEYS[kv[0]][0]: OSD_KEYS[kv[0]][1](kv[1]) for kv in (
            line.split(': ') for line in osd.split('\n')
        ) if len(kv) == 2 and is_valid(kv[1], OSD_KEYS[kv[0]][1])
    }

In [0]:
@run_once
def get_tesseract_version():
    '''
    Returns LooseVersion object of the Tesseract version
    '''
    try:
        return LooseVersion(
            subprocess.check_output(
                [tesseract_cmd, '--version'], stderr=subprocess.STDOUT
            ).decode('utf-8').split()[1].lstrip(string.printable[10:])
        )
    except OSError:
        raise TesseractNotFoundError()


In [0]:
def image_to_string(image,
                    lang=None,
                    config='',
                    nice=0,
                    output_type=Output.STRING):
    '''
    Returns the result of a Tesseract OCR run on the provided image to string
    '''
    args = [image, 'txt', lang, config, nice]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: {'text': run_and_get_output(*args)},
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


In [0]:
def image_to_pdf_or_hocr(image,
                    lang=None,
                    config='',
                    nice=0,
                    extension='pdf'):
    '''
    Returns the result of a Tesseract OCR run on the provided image to pdf/hocr
    '''

    if extension not in {'pdf', 'hocr'}:
        raise ValueError('Unsupported extension: {}'.format(extension))
    args = [image, extension, lang, config, nice, True]

    return run_and_get_output(*args)

In [0]:
def image_to_boxes(image,
                   lang=None,
                   config='',
                   nice=0,
                   output_type=Output.STRING):
    '''
    Returns string containing recognized characters and their box boundaries
    '''
    config += ' batch.nochop makebox'
    args = [image, 'box', lang, config, nice]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: file_to_dict(
            'char left bottom right top page\n' + run_and_get_output(*args),
            ' ',
            0),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()

In [0]:
def get_pandas_output(args):
    if not pandas_installed:
        raise PandasNotSupported()

    return pd.read_csv(
        BytesIO(run_and_get_output(*args)),
        quoting=QUOTE_NONE,
        sep='\t'
    )

In [0]:
def image_to_data(image,
                  lang=None,
                  config='',
                  nice=0,
                  output_type=Output.STRING):
    '''
    Returns string containing box boundaries, confidences,
    and other information. Requires Tesseract 3.05+
    '''

    if get_tesseract_version() < '3.05':
        raise TSVNotSupported()

    config = '{} {}'.format('-c tessedit_create_tsv=1', config.strip()).strip()
    args = [image, 'tsv', lang, config, nice]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DATAFRAME: lambda: get_pandas_output(args + [True]),
        Output.DICT: lambda: file_to_dict(run_and_get_output(*args), '\t', -1),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


In [0]:
def image_to_osd(image,
                 lang='osd',
                 config='',
                 nice=0,
                 output_type=Output.STRING):
    '''
    Returns string containing the orientation and script detection (OSD)
    '''
    config = '{}-psm 0 {}'.format(
        '' if get_tesseract_version() < '3.05' else '-',
        config.strip()
    ).strip()
    args = [image, 'osd', lang, config, nice]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: osd_to_dict(run_and_get_output(*args)),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()

In [0]:
def main():
    if len(sys.argv) == 2:
        filename, lang = sys.argv[1], None
    elif len(sys.argv) == 4 and sys.argv[1] == '-l':
        filename, lang = sys.argv[3], sys.argv[2]
    else:
        sys.stderr.write('Usage: python pytesseract.py [-l lang] input_file\n')
        exit(2)

    try:
        with Image.open(filename) as img:
            print(image_to_string(img, lang=lang))
    except IOError:
        sys.stderr.write('ERROR: Could not open file "%s"\n' % filename)
        exit(1)

In [0]:
if __name__ == '__main__':
    main()

## VI. Transpose
Transpose the vertically scanned result to horizontal

## VII. Quality Check
Match the transposed vertical text file with horizontally scanned result to quality check

## VIII. Structure Selection & Data Categorization
Prompts the user for structure (formatting and organization of data)

## IX. Reassemble Data
Create a CSV or Excel file with the processed data

In [52]:
! pip install XlsxWriter
! pip install pandas
import pandas as pd

    100% |████████████████████████████████| 143kB 6.6MB/s 


In [0]:
df= pd.DataFrame(split)
#df = df.transpose()

print(df)

In [0]:
df.to_excel("output.xlsx")